# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import datetime
import tqdm

#### <span style="color:green">Plot functions</span>

In [3]:
variables_dict = {'speed': 'Velocidad', 'latitude':'Lat', 'longitude':'lng', 'accPosition': 'Presión del acelerador',
                  'accX': 'Aceleración en X', 'accY': 'Aceleración en Y', 'accZ': 'Aceleración en Z',
                  'magX': 'Fuerza magnética en X', 'magY': 'Fuerza magnética en Y', 'magZ': 'Fuerza magnética en Z',
                  'velAngX': 'Velocidad angular en X', 'velAngY': 'Velocidad angular en Y', 'velAngZ': 'Velocidad angular en Z'}
units_dict = {'speed': 'm/s', 'latitude':'°', 'longitude':'°', 'accPosition': '% de presión',
              'accX': 'm/s\u00B2', 'accY': 'm/s\u00B2', 'accZ': 'm/s\u00B2',
              'magX': '\u03BC T', 'magY': '\u03BC T', 'magZ': '\u03BC T',
              'velAngX': 'rad/seg', 'velAngY': 'rad/seg', 'velAngZ': 'rad/seg'}


def update_layout(fig, chart_title: str, **kwargs):
    """Update layout for every chart

    Args:
        fig: A plotly figure
        chart_title (str): A title for chart

    Returns:
        fig: the updated layout for the plotly figure
    """
    # Get kwards data if is needed
    x_title = kwargs.get('xaxis_title', None)
    y_title = kwargs.get('yaxis_title', None)
    height_size = kwargs.get('height', 450)

    # Update layout of the figure
    fig.update_layout(
        title={'text': chart_title, 'x': 0.5},
        legend_title='Tipo de evento',
        xaxis_title=x_title,
        yaxis_title=y_title,
        template='plotly_white',
        autosize=True,
        height=height_size,
        font=dict(
            family="BlinkMacSystemFont,-apple-system,Segoe UI,Roboto,Oxygen,Ubuntu, \
                    Cantarell,Fira Sans,Droid Sans,Helvetica Neue,Helvetica,Arial,sans-serif",
            size=14,
            color="#363636"
        )
    )

    return fig


def line_chart(df, variable):
    """Line chart created with plotly graphic object

    Args:
        df (DataFrame): The dataframe that contains the data to plot
        variable (str): The specific variable to plot

    Returns:
        fig: A plotly figure
    """
    # Create a copy of the data frame to avoid losing important data
    near_crash_df = df.copy()
    no_crash_df = df.copy()
    x = df['timestamp']
    # Filter colums for eventClass
    # Set all variables where have a normal events to NaN
    near_crash_df.loc[df['eventClass'] == 0, variable] = None
    # Set all variables where have a near-crash events to NaN
    no_crash_df.loc[df['eventClass'] == 1, variable] = None

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=no_crash_df[variable],
        name='Sin evento',
        marker_color='#00D1B1',
        text=no_crash_df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))
    fig.add_trace(go.Scatter(
        x=x,
        y=near_crash_df[variable],
        name='Near-Crash',
        marker_color='#FF385F',
        text=near_crash_df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))

    fig = update_layout(fig, f'Gráfico de linea de la {variables_dict[variable]}',
                        xaxis_title='Marca de Tiempo', yaxis_title=units_dict[variable])
    fig.update_yaxes(nticks=12)
    fig.update_xaxes(nticks=12, tickangle=45)

    return fig


### Cargar datos

In [4]:
df = pd.read_csv("./data/smartphone_02-Feb-2022_id2_aceleración repentina_Data-Muy3Q2oTXSwbX4Xkp8f.csv")
df.head(3)

,id,accX,accY,accZ,eventClass,idTrip,idVehicle,latitude,longitude,magX,magY,magZ,route,speed,timestamp,velAngX,velAngY,velAngZ
0,31054,0.474078,1.481456,9.672387,0,2,Kia Picanto (Xiaomi Redmi Note 9S),0.834703,-77.649896,30.037500,2.70000,32.081253,Aceleración repentina,17.712,2022-02-02 19:57:31.294000-05:00,0.000000,0.001465,-0.000533
1,31055,0.443570,1.491626,9.634102,0,2,Kia Picanto (Xiaomi Redmi Note 9S),0.834703,-77.649896,29.981251,2.58750,32.156250,Aceleración repentina,17.712,2022-02-02 19:57:31.343000-05:00,0.000533,-0.000399,-0.000799
2,31056,0.442074,1.474876,9.640682,0,2,Kia Picanto (Xiaomi Redmi Note 9S),0.834703,-77.649896,29.906252,2.68125,32.343750,Aceleración repentina,17.712,2022-02-02 19:57:31.393000-05:00,0.000399,-0.000266,-0.000133


# Limpieza de datos

Para estos han sido identificadas ciertas condiciones a satisfacer:

### Eliminar o agregar eventos de near-crash
Primero necesitamos encajar los eventos de *near-crash* donde corresponden, eliminando o agregando en la columna **eventClass** un 
<span style="color:red">"1"</span> si corresponde a un evento o un <span style="color:cyan">"0"</span> si no corresponde a este.

In [5]:
print("RAW DATA")
line_chart(df, "accY")

RAW DATA


In [6]:
min_ID = 31292  # ID to start near-crash
max_ID = 31347  # ID to end near-crash

df.loc[(df["id"] >= min_ID) & (df["id"] <= max_ID), "eventClass"] = 1
print("Offset correction")
line_chart(df, "accY")

### Interpolar algunas variables

La interpolación nos permite tener un muestreo general en los datos, obteniendo por ejemplo mas puntos en la variable de la velocidad. *(Solo en el caso de la Raspberry para la velocidad)*

### Corregir tiempos y offset en algunas variables
Algunos variables cineamticas estan desplazadas verticalmente por lo tanto necesitamos corregir este desface.

In [7]:
max_standby = 31291
df["accY"] = df["accY"] - df.loc[df['id'] <= max_standby, "accY"].mean()
line_chart(df, "accY")

#### Check final results

In [15]:
# All variables: speed	accX	accY	accZ	magX	magY	magZ	velAngX	velAngY	velAngY
#                eventClass	idTrip	idVehicle	latitude	longitude	route	id	timestamp
variables = ["speed", "accX", "accY", "accZ", "velAngX", "velAngY", "velAngZ", "magX", "magY", "magZ"]
for var in variables:
    fig = line_chart(df, var)
    fig.show()